# Technical challenge
## Nubimetrics – Data Engineering Jr

Solución del *Technical challenge* realizada por *Juan Matías Mascioto*.

A partir del dataset de ventas de teléfonos móviles (*data/raw/cellphoneslisting.csv*), la intención es poder detectar la **marca** de cada venta a partir de su título (*title*). Luego, una vez detectada la marca, se agruparán las ventas para poder sumarizar las cantidades totales de ventas en dinero (*sales*) y unidades (*unit_sales*).

Consideraciones:
 * Se utilizó un dataset (*data/raw/phone_dataset.csv*) con datos de teléfonos móviles descargados de [GSMArena.com](https://www.gsmarena.com/)
  * [Link al dataset](https://www.kaggle.com/arwinneil/gsmarena-phone-dataset)
  * La performance de la detección de las marcas está sujeta a la fiabilidad de estos datos. En caso que se quiera mejorar la performance se debería buscar/armar un dataset más completo.
 * La resolución implica cierto conocimiento del dominio (*teléfonos móviles*). Por ejemplo, saber que *Moto* y *Motorola* se usan indistintamente, o que *iPhone* pertenece a *Apple*.

In [1]:
# Imports
import jellyfish
import pandas as pd
import re
from collections import Counter
from IPython.display import display, Markdown as md

In [2]:
# Config
REPLACEMENTS = {"MOTO": "MOTOROLA", "IPHONE": "APPLE"} # Reemplazo de palabras para mejorar la performance

## Carga de los datos
Se leen los datos de los archivos *.csv*. El listado de ventas se guarda en *df*, en un *DataFrame* de *pandas*. Por otra parte, el listado de marcas se guarda en una lista en *brands*.

In [3]:
df = pd.read_csv("data/raw/cellphoneslisting.csv", sep="\x01")
brands = pd.read_csv("data/raw/phone_dataset.csv", sep=",", usecols=["brand"]).drop_duplicates()["brand"]\
    .str.upper().tolist()

## Exploración de los datos
Se verifica que los datos se hayan cargado correctamente en el *DataFrame* y vemos las dimensiones del mismo.

In [4]:
display(df.head())
display(md("**Filas**: {0} - **Columnas**: {1}".format(df.shape[0], df.shape[1])))

,id,title,sales,unit_sales
0,1,Nokia Asha 503 Dual Sim Liberado Nuevos Gar...,2996.99,1
1,2,Samsung Galaxy A5 A500 16gb Refabricado 2gb Ra...,31495.00,5
2,3,Celular Android Smartphone Liberado 4g Lte Tcl...,77380.00,20
3,4,Telefono Celular Lg G7 4g Lte Libre Nuevo Gara...,99997.00,3
4,5,Telefono Celular Lg G7 New Platinum Gray 4g Lt...,219992.00,8


**Filas**: 4047 - **Columnas**: 4

También chequeamos que los datos en *brands* se hayan cargado correctamente.

In [5]:
display(md("Cantidad de marcas en *brands*: {}".format(len(brands))))
display(brands)

Cantidad de marcas en *brands*: 108

['ACER',
 'ALCATEL',
 'ALLVIEW',
 'AMAZON',
 'AMOI',
 'APPLE',
 'ARCHOS',
 'ASUS',
 'AT&T',
 'BENEFON',
 'BENQ',
 'BENQ-SIEMENS',
 'BIRD',
 'BLACKBERRY',
 'BLU',
 'BOSCH',
 'BQ',
 'CASIO',
 'CAT',
 'CELKON',
 'CHEA',
 'COOLPAD',
 'DELL',
 'EMPORIA',
 'ENERGIZER',
 'ERICSSON',
 'ETEN',
 'FUJITSU SIEMENS',
 'GARMIN-ASUS',
 'GIGABYTE',
 'GIONEE',
 'GOOGLE',
 'HAIER',
 'HP',
 'HTC',
 'HUAWEI',
 'I-MATE',
 'I-MOBILE',
 'ICEMOBILE',
 'INNOSTREAM',
 'INQ',
 'INTEX',
 'JOLLA',
 'KARBONN',
 'KYOCERA',
 'LAVA',
 'LEECO',
 'LENOVO',
 'LG',
 'MAXON',
 'MAXWEST',
 'MEIZU',
 'MICROMAX',
 'MICROSOFT',
 'MITAC',
 'MITSUBISHI',
 'MODU',
 'MOTOROLA',
 'MWG',
 'NEC',
 'NEONODE',
 'NIU',
 'NOKIA',
 'NVIDIA',
 'O2',
 'ONEPLUS',
 'OPPO',
 'ORANGE',
 'PALM',
 'PANASONIC',
 'PANTECH',
 'PARLA',
 'PHILIPS',
 'PLUM',
 'POSH',
 'PRESTIGIO',
 'QMOBILE',
 'QTEK',
 'SAGEM',
 'SAMSUNG',
 'SENDO',
 'SEWON',
 'SHARP',
 'SIEMENS',
 'SONIM',
 'SONY',
 'SONY ERICSSON',
 'SPICE',
 'T-MOBILE',
 'TEL.ME.',
 'TELIT',
 'THURA

In [6]:
# df = df.iloc[0:100] # Descomentar esta línea para testear el programa con un subset de los datos

## Normalización
Se normaliza el campo *title* para simplificar las comparaciones de strings.

In [7]:
def normalize(s):
    """ Retorna 's' normalizado """
    s = s.upper()   # Convierte a mayúsculas
    s = re.sub("[^0-9a-zA-Z]+", " ", s) # Reemplaza caracteres no alfanuméricos por espacios en blanco

    return s

df["title_normalized"] = df["title"].apply(normalize)

display(df.head())

,id,title,sales,unit_sales,title_normalized
0,1,Nokia Asha 503 Dual Sim Liberado Nuevos Gar...,2996.99,1,NOKIA ASHA 503 DUAL SIM LIBERADO NUEVOS GARANTIA
1,2,Samsung Galaxy A5 A500 16gb Refabricado 2gb Ra...,31495.00,5,SAMSUNG GALAXY A5 A500 16GB REFABRICADO 2GB RA...
2,3,Celular Android Smartphone Liberado 4g Lte Tcl...,77380.00,20,CELULAR ANDROID SMARTPHONE LIBERADO 4G LTE TCL...
3,4,Telefono Celular Lg G7 4g Lte Libre Nuevo Gara...,99997.00,3,TELEFONO CELULAR LG G7 4G LTE LIBRE NUEVO GARA...
4,5,Telefono Celular Lg G7 New Platinum Gray 4g Lt...,219992.00,8,TELEFONO CELULAR LG G7 NEW PLATINUM GRAY 4G LT...


## Extracción de la marca
Para determinar la **marca** a partir del título del producto se van a comparar todas las palabras del título (*title*) contra todas las marcas en *brands*. Esta comparación genera una matriz de distancias, donde el mínimo valor indicará cuál es la marca.

Se utiliza como distancia de edición la [Distancia de Damerau-Levenshtein](https://es.wikipedia.org/wiki/Distancia_de_Damerau-Levenshtein). Esta distancia (a diferencia de la Distancia de [Levenshtein](https://es.wikipedia.org/wiki/Distancia_de_Levenshtein)) penaliza a la transposición de caracteres como una única operación de edición. La transposición de caracteres suele ser un error frecuente en textos tipeados manualmente, y por ese motivo se utiliza esta distancia.
Además, la distancia se normaliza a valores entre 0 y 1 para que la métrica considere la longitud de las palabras. Un *match* exacto obtiene una distancia 0, mientras que la máxima distancia es 1.

In [8]:
def get_brand(s):
    """ Retorna la marca extraída del título normalizado, la marca de la lista de marcas y la distancia """
    # Separa las palabras y las reemplaza si están en 'REPLACEMENTS'
    words = [w if w not in REPLACEMENTS.keys() else REPLACEMENTS[w] for w in s.split()]
    words = list(set(words))  # Borra palabras duplicadas (no tiene sentido compararlas más de una vez)
    
    # Matriz de distancias entre cada una de las palabras en 'title' (filas) y cada una de las marcas en 'brands' (columnas)
    dist_matrix = pd.DataFrame(index=words, columns=brands)
    for w in words:
        dist_matrix.loc[w] = [jellyfish.damerau_levenshtein_distance(w, b) / max(len(w), len(b)) for b in brands]

    return dist_matrix.min(axis=1).idxmin(), dist_matrix.min(axis=0).idxmin(), dist_matrix.min(axis=1).min()

for index, title_normalized in df["title_normalized"].iteritems(): # Itera por cada uno de los títulos normalizados
    brand_from_title, brand_from_list, dist = get_brand(title_normalized)
    df.loc[index, "brand_title"] = brand_from_title
    df.loc[index, "brand_list"] = brand_from_list
    df.loc[index, "dist"] = dist
    
display(df.head())

,id,title,sales,unit_sales,title_normalized,brand_title,brand_list,dist
0,1,Nokia Asha 503 Dual Sim Liberado Nuevos Gar...,2996.99,1,NOKIA ASHA 503 DUAL SIM LIBERADO NUEVOS GARANTIA,NOKIA,NOKIA,0.000000
1,2,Samsung Galaxy A5 A500 16gb Refabricado 2gb Ra...,31495.00,5,SAMSUNG GALAXY A5 A500 16GB REFABRICADO 2GB RA...,SAMSUNG,SAMSUNG,0.000000
2,3,Celular Android Smartphone Liberado 4g Lte Tcl...,77380.00,20,CELULAR ANDROID SMARTPHONE LIBERADO 4G LTE TCL...,LTE,ZTE,0.333333
3,4,Telefono Celular Lg G7 4g Lte Libre Nuevo Gara...,99997.00,3,TELEFONO CELULAR LG G7 4G LTE LIBRE NUEVO GARA...,LG,LG,0.000000
4,5,Telefono Celular Lg G7 New Platinum Gray 4g Lt...,219992.00,8,TELEFONO CELULAR LG G7 NEW PLATINUM GRAY 4G LT...,LG,LG,0.000000


Si vemos rápidamente la tabla anterior, podemos notar lo siguiente:
 * En las filas con *id* 1, 2, 4 y 5 encuentra las marcas perfectamente (*NOKIA*, *SAMSUNG* y *LG*), y la distancia es 0.
 * En la fila con *id* 3 hay un error. Confunde *LTE* con la marca *ZTE*, y la distancia es 0.33.
 
Miremos un poco más detalladamente los casos en los que no hubo coincidencias exactas (*dist* mayor a 0).

In [9]:
df_errores = df.query("dist > 0").sort_values(by="dist")

display("Filas sin match exacto: {0} ({1}%)".format(df_errores.shape[0], round(100 * df_errores.shape[0]/df.shape[0], 2)))
display(df_errores.loc[:, ["title_normalized", "brand_title", "brand_list", "dist"]])

'Filas sin match exacto: 400 (9.88%)'

,title_normalized,brand_title,brand_list,dist
307,BLACBERRY CURVE 9300 WI FI 3G IMPECABLES MOVISTAR,BLACBERRY,BLACKBERRY,0.100000
2635,MOTOROLLA E2 LIBERADO CON GARANTIA,MOTOROLLA,MOTOROLA,0.111111
3760,SAMGUNG J7 PRO ORIGINAL 64GB LIBRE DE FABRICA,SAMGUNG,SAMSUNG,0.142857
2696,SAMSUMG J8 2018,SAMSUMG,SAMSUNG,0.142857
844,CELULAR ALKATEL ONETOUCH 1052D TECLAS GRANDES,ALKATEL,ALCATEL,0.142857
2871,SAMSUN GALAXY J8 32 GB 3 GB RAM,SAMSUN,SAMSUNG,0.142857
233,SANSUNG GALAXY CORBY GT S3650 PERSONAL,SANSUNG,SAMSUNG,0.142857
3222,TELEFONO CELULAR SAMUNG J7 PRIME 16GB OFERTA,SAMUNG,SAMSUNG,0.142857
1679,XIOAMI REDMI NOTE 6 PRO 64 GB 4 RAM GLOBAL NUEVO,XIOAMI,XIAOMI,0.166667
2850,XIOAMI MI A2 DE 64GB GTIA 12 MESES FACT,XIOAMI,XIAOMI,0.166667


Se puede ver claramente que a partir de *dist* mayores o iguales a 0.2 los resultados no son confiables. Vamos a asignarle la marca *OTROS* a todos estos casos.

In [10]:
THRESHOLD = 0.2 # Umbral para determinar cuándo un match es 'confiable'

df.loc[df["dist"] >= THRESHOLD, "brand_list"] = "OTROS"

## Agrupamiento por marca
A partir del campo *brand_list* podemos agrupar las ventas por **marca**, y calcular las ventas totales.

In [11]:
df_grouped = df.groupby(by="brand_list").sum().loc[:, ["unit_sales", "sales"]].sort_values(by="unit_sales", ascending=False)

display(df_grouped)

,unit_sales,sales
brand_list,,
SAMSUNG,8748,86019101.00
MOTOROLA,5651,55593865.59
LG,5287,34413645.32
XIAOMI,5085,60845433.31
OTROS,3572,15049774.59
HUAWEI,1611,22062052.84
BLU,863,2576862.21
NOKIA,640,1679269.18
APPLE,544,14744294.21


In [12]:
# Verificamos que los totales coincidan
assert df["sales"].sum() == df_grouped["sales"].sum(), "Error en sales"
assert df["unit_sales"].sum() == df_grouped["unit_sales"].sum(), "Error en unit_sales"

## Resultados
Se guardan los resultados en el archivo *data/output/ventas_por_marca.csv*. Además, en el archivo *data/output/cellphoneslisting_processed.csv* se guarda el listado de ventas con la información adicional generada en este proyecto.

In [13]:
df_grouped.to_csv("data/output/ventas_por_marca.csv", sep=";", index=True)
df.to_csv("data/output/cellphoneslisting_processed.csv", sep=";", index=False)

## Comentarios finales
 * Es posible reducir la cantidad de items clasificados como *OTROS* si se enriquece el dataset de marcas. Revisando los títulos de los productos que actualmente están clasificados como *OTROS* se pueden encontrar marcas para agregar.
 * Se podría haber generado un set de marcas propio y no utilizando un dataset externo. Si se arma una colección con la cantidad de veces que aparece cada palabra en los títulos, es esperable que las marcas sean de las palabras más repetidas (junto con otras palabras genéricas como *celular*, *telefono*, *libre*, por ejemplo). Es posible que esta solución sea más indicada para utilizar en otros dominios de datos, donde encontrar un dataset resulte más díficil. A continuación se muestra un pequeño ejemplo (sin depurar) de cómo se podría implementar esta solución.

In [14]:
TRASH_REGEX = "CELULAR|DUAL|LIBERADO|LIBRE|NUEVO|PLUS|RAM|SIM|TELEFONO|\d+G|\d" # Regex para descartar palabras

word_counter = Counter()
for index, title_normalized in df["title_normalized"].iteritems():
    words = [w for w in title_normalized.split() if re.search(TRASH_REGEX, w) is None]
    word_counter += Counter(words)

print("10 palabras más repetidas: {}".format(word_counter.most_common(10)))

10 palabras más repetidas: [('SAMSUNG', 963), ('XIAOMI', 792), ('GALAXY', 665), ('MOTO', 583), ('MOTOROLA', 574), ('LITE', 384), ('REDMI', 365), ('MI', 341), ('A', 340), ('GTIA', 324)]
